In [1]:
import numpy as np
import cortex

In [21]:
comps = np.load('components.npz')['arr_0']
comps = comps.transpose(0,3,2,1)
comps.reshape(10,-1).shape

(10, 902629)

In [22]:
def make_colorvol(proj, meanstds, rgbpcs, mask, flips=tuple(), clip_lim=3):
    rgbdata = proj[rgbpcs,:]
    for f in flips:
        rgbdata[f] *= -1
    zrgb = (rgbdata.T / meanstds).T
    #zrgb = npp.rs(rgbdata.T).T
    crgb = np.clip(zrgb, -clip_lim, clip_lim) ## clip to 3 standard deviations
    srgb = crgb/clip_lim/2.0 + 0.5 ## scale to 0..1
    
    rgbvol = np.zeros([3]+list(mask.shape))
    rgbvol[0] = vox_to_mask(srgb[0], mask)
    rgbvol[1] = vox_to_mask(srgb[1], mask)
    rgbvol[2] = vox_to_mask(srgb[2], mask)

    colorvol = (rgbvol*255).astype(np.uint8)
    return colorvol.transpose(1,2,3,0)

def vox_to_mask(data, mask):
    dvol = mask.copy().astype(np.float)
    dvol[mask>0] = data
    return dvol


In [23]:
rescale = lambda x: x/x.std()

In [32]:
flat_comps = comps.reshape(10,-1)

colorvol = make_colorvol(flat_comps, flat_comps.std(1)[:3], [7,8,9], np.ones(comps[0].shape), clip_lim=4)

In [33]:
cortex.webshow((colorvol, 'fsaverage_pycortex', 'atlas_2mm'))

Started server on port 47008


<JS: window.viewer>